# ERA5 & MERIT: Monthly Potential Runoff Export

This data set is large because MERIT is very high resolution.


In [1]:
import ee

import sys
sys.path.insert(1, '../SMAP-USDA/')  # until we package this
import mon_stats
import water_common

In [2]:
# import for ipynb
%run era5_merit_mon_avg_runoff.ipynb

In [3]:
month_names = mon_stats.month_names

In [4]:
# TODO: 
# 1. Global is 90GB images, 
#   * masking larger values can reduce this but maybe not alot
#   * total asset size <=250GiB, not sure that's a single asset or all assets, 
#     will try exporting these to individual images
# 2. The band is still 'upa' in the output image
bbox_dict = water_common.bboxes()
bbox_name = 'conus'
bbox = bbox_dict[bbox_name]
variables = ['potential_sfc_runoff_mon_clim_' + 
             bbox_name + '_cms']
collection_name = (
    'users/jamesmcc/era5_merit_potential_sfc_runoff_' 
    + bbox_name + '_climatology')
runoff_asset = ee.data.createAsset(
    {'type': 'ImageCollection'}, collection_name)
# runoff_asset = {'id': collection_name}
max_pixels = 100000000000  #For global is quite large

# Mask here? or before?

for vv in variables:
    for mm0 in range(12):
        mm = mm0 + 1
        mon_name = month_names[mm0]
        description = mon_name
        xx = ee.batch.Export.image.toAsset(
            ee.Image(potential_sfc_runoff_clim_cms.get(mon_name)),
            description=description, 
            assetId=runoff_asset['id'] + '/' + description,
            region=bbox, 
            maxPixels=max_pixels).start()